In [16]:
from google.colab import drive, files
drive.mount('/content/drive')

%cd "./drive/My Drive/Colab Notebooks/neural-learning/src"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: './drive/My Drive/Colab Notebooks/neural-learning/src'
/content/drive/My Drive/Colab Notebooks/neural-learning/src


In [0]:
import json
import numpy as np
import torch
import os
from scipy.signal import find_peaks

from lif import lif_compute, spike_binary, id_synaptic_waveform
from ou_process import ouprocess_gaussian
from spike_sync import find_synced_spikes

In [0]:
torch.autograd.set_grad_enabled(False)

NUM_NEURONS = 20 # 200
tau_V = 10
R = 1 # MOhm
EL = -70.0
V_th = -40.0
dt = 0.1 # msec
t_stop = 3.0e3 # 0.0e3

std_noise = 25

W = torch.zeros(NUM_NEURONS, 1)
train_input = None
train_exp_output = None

v_E = 0.0
v_ave = -67.0

tau_rise = 0.5
tau_fall = 5.0
syn_kernel_len = 50.0

_ETA, _ = ouprocess_gaussian(5.0, dt, t_stop, NUM_NEURONS)

In [0]:
torch.autograd.set_grad_enabled(False)

input_slow, _ = ouprocess_gaussian(50.0, dt, t_stop, 1)
i_inj = 16.0 + 6.0*input_slow

In [0]:
tt = np.arange(0.0, t_stop, dt)
V = np.zeros((tt.shape[0], NUM_NEURONS)) # Membrane potential per neuron

# Additive noise to individual neurons
if _ETA is None \
        or int_noise_regen is True\
        or _ETA.shape != V.shape:
    _ETA, _ = ouprocess_gaussian(5.0, dt, t_stop, NUM_NEURONS)

F_binary = np.zeros((tt.shape[0], NUM_NEURONS))
# avg_firing_rate = np.zeros(self.NUM_NEURONS)

I_total = std_noise*_ETA + i_inj

V = lif_compute(I_total, R, tau_V, V_th, dt)
F_binary = spike_binary(V)

syn_waveform = id_synaptic_waveform(
    dt,
    syn_kernel_len,
    tau_rise,
    tau_fall)
syn_wave_len = syn_waveform.shape[0]
t_steps = F_binary.shape[0]

F_synaptic = np.zeros(F_binary.shape)
for neuron in range(0, NUM_NEURONS):
    fr_fast = np.convolve(F_binary[:,neuron], syn_waveform)
    F_synaptic[:, neuron] = fr_fast[:-syn_wave_len+1]

ind_neur = np.arange(0, NUM_NEURONS)
Phi = F_synaptic[:t_steps, ind_neur]
X2 = -1.0*v_ave*np.ones((t_steps,ind_neur.shape[0])) + v_E

In [0]:
A = np.multiply(Phi, X2)
out = np.dot(A, W)

In [0]:
out.shape

(300000, 200)

In [0]:
torch.autograd.set_grad_enabled(False)

I_total = std_noise*_ETA + i_inj

In [28]:
_ETA

(array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.18916729, -0.47801467, -0.27067824, ..., -0.11892605,
          0.00917254, -0.0628251 ],
        ...,
        [ 0.34650853, -0.40019192,  0.18686829, ...,  0.60566231,
         -0.85457788, -0.65818338],
        [ 0.54210838, -0.51726354,  0.70101179, ...,  0.74274031,
         -0.54970171, -0.4232218 ],
        [ 0.49460049, -0.25204329,  0.69253153, ...,  0.55064781,
         -0.74005255, -0.19734656]]),
 array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.95532633, -2.41405374, -1.36697025, ..., -0.60059639,
          0.04632285, -0.31727721],
        [-0.89775695, -1.34804312,  1.66316257, ...,  0.2707488 ,
         -1.09949692,  1.37745395],
        ...,
        [ 1.02246268, -0.63125041,  2.61519718, ...,  

In [50]:
torch.autograd.set_grad_enabled(False)

EL = -70.0  # mV
# V_th = Th  # -50
V = torch.ones(I_total.shape[1])*EL

V_reset = -90.0

k = 0
total_time = len(I_total) - 1.0/dt
V_out = torch.zeros(I_total.shape)

spike_reset_count = torch.zeros(I_total.shape[1],1)

while k <= total_time:
    # LIF equation
    print("Time: ", k)
    dq_dt = -1.0 * (V - EL)
    add = float(R) * torch.as_tensor(I_total[k]).float()
    dq_dt = dq_dt + add
    dq_dt = 1.0 / tau_V * dq_dt
    V = V + dt*dq_dt

    # Spike
    spike = 50.0*torch.ge(V, V_th).float() # Spike
    spike_reset_add = ((int(1.0/dt)+1.0)*torch.ge(V, V_th))
    spike_reset_count = torch.add(spike_reset_count, spike_reset_add[:,None].float())

    # No spike
    subthresh = torch.mul(torch.eq(spike_reset_count, 0.0).float(), V[:,None].float())
    reset = torch.mul(torch.gt(spike_reset_count, 0.0).float(), V_reset)
    reset_or_subthres = reset + subthresh
    no_spike = torch.mul(reset_or_subthres, torch.lt(V, V_th)[:,None].float()).flatten()

    V_out[k] = spike + no_spike
    V = torch.mul(V_out[k,:], torch.eq(spike_reset_count, 0.0).float().flatten()) + reset.flatten()

    spike_reset_count = spike_reset_count - 1.0*torch.gt(spike_reset_count, 0.0).float()

    k += 1

    # # Spike
    # if np.greater_equal(V, V_th): # V >= V_th:
    #     V_out[k] = 50.0  # TODO: ???
    #     V_out[k+1:int(k+1.0/dt)+1] = V_reset # TODO: ???
    #     0:10+1 -> 11 steps

    #     k += int(1.0/dt) # TODO: ???
    # # No spike
    # else:
    #     V_out[k] = V
    #     k += 1
    # V = V_out[k-1]

while k < V_out.shape[0]:
    V_out[k] = V
    k += 1

# return V_out.numpy()


Time:  0
Time:  1
Time:  2
Time:  3
Time:  4
Time:  5
Time:  6
Time:  7
Time:  8
Time:  9
Time:  10
Time:  11
Time:  12
Time:  13
Time:  14
Time:  15
Time:  16
Time:  17
Time:  18
Time:  19
Time:  20
Time:  21
Time:  22
Time:  23
Time:  24
Time:  25
Time:  26
Time:  27
Time:  28
Time:  29
Time:  30
Time:  31
Time:  32
Time:  33
Time:  34
Time:  35
Time:  36
Time:  37
Time:  38
Time:  39
Time:  40
Time:  41
Time:  42
Time:  43
Time:  44
Time:  45
Time:  46
Time:  47
Time:  48
Time:  49
Time:  50
Time:  51
Time:  52
Time:  53
Time:  54
Time:  55
Time:  56
Time:  57
Time:  58
Time:  59
Time:  60
Time:  61
Time:  62
Time:  63
Time:  64
Time:  65
Time:  66
Time:  67
Time:  68
Time:  69
Time:  70
Time:  71
Time:  72
Time:  73
Time:  74
Time:  75
Time:  76
Time:  77
Time:  78
Time:  79
Time:  80
Time:  81
Time:  82
Time:  83
Time:  84
Time:  85
Time:  86
Time:  87
Time:  88
Time:  89
Time:  90
Time:  91
Time:  92
Time:  93
Time:  94
Time:  95
Time:  96
Time:  97
Time:  98
Time:  99
Time:  100